### This section is for functions I am going to need and all data processing down to a d_ci dataframe. That frame represents data for clustering. Changes are imminent, but this block sets everything up for analysis

Consider breaking out further

In [1]:
import os
import re
import math
import config
import kmeans
import extract
import constants
import matplotlib as mpl
import numpy as np
import pandas as pd
import transform
import plotly as py
import sys

from scipy.stats import zscore
from sklearn.metrics import pairwise

#reload(sys)
py.tools.set_credentials_file(username='luci1408', api_key='dME8WYW9UKDs7cNWXPKm')

mpl.use('TkAgg')

files_location = config.data_source_file_location
files = os.listdir(files_location)

###############################################
# Functions which will eventually find a better home
#
pd.Series.filter_by = kmeans.filter_by
pd.DataFrame.filter_by = kmeans.filter_by


# Now we must clean the data for modeling. Think about recording transformations and informationa bout them

Cluster hotels within market by:
    -Distance to other hotels (how many hotels are within an x radius of this one)
    -Operating Revenue 'ANSWER_ann_op_rev'
    -Size of meeting space or meetings related 'ANSWER_sq_ft_meeting'
    -Number of Rooms 'ANSWER_num_rooms'
    -RevPAR 'ANSWER_ann_revenue/ANSWER_num_rooms*365'
    -STR Chain Scale? Diamonds? Stars?

In [2]:
# Register transform functions here, create them in the transform.py
column_operations = {'ANSWER_num_rooms':transform.answer_num_rooms,
                     'ANSWER_ann_op_rev':transform.answer_ann_op_rev,
                     'ANSWER_ann_revenue':transform.answer_ann_revenue,
                     'ANSWER_num_employees_pos':transform.answer_num_employees_pos,
                     'ANSWER_cur_base_pay':transform.answer_cur_base_pay,
                     'create_RevPAR':transform.create_RevPAR,
                     'create_Executive':transform.create_Executive}


d_ci = extract.extract_all_lazy()
d_ci_v2  = transform.clean_model(d_ci, column_operations)

#Final preprocessing step before modeling. Normalize numeric score via Z-score
features_norm = [f for f in column_operations.keys() if 'ANSWER' in f]
d_ci_v2[features_norm] = d_ci_v2[features_norm].convert_objects(convert_numeric=True)#.apply(zscore)

# The top X number of markets by the number of properties
number_of_markets = 15
number_of_clusters = 3
markets = kmeans.top_markets_by_property(number_of_markets, d_ci_v2)
positions = d_ci_v2['POSITION'].unique().tolist()

modeling = d_ci_v2[features_norm[0:4]]

AttributeError: 'module' object has no attribute 'clean_hotel_model'

In [ ]:
d_ci.columns

In [ ]:
import sys
import numpy as np
import random
#import mykmns
#import readdata
import extract
from sklearn import cluster
from scipy.spatial import distance_matrix

#------------Input---------------
# n = 2000 # total number of points
# cluster_size = 4 # size of each cluster
# k = (n+cluster_size-1) // cluster_size # total number of clusters
n = modeling.shape[0]
k = 4
infl = 't4p2k.pdb' # the input pdb file with all dataset
outfl = 'cluster2k.pdb' # the output pdb file with the clustering information
boxl = np.array([39.0571])#, 39.0571])
max_iter = 100

def filter_by(df, constraints):
    """Filter MultiIndex by sublevels."""
    indexer = [constraints[name] if name in constraints else slice(None)
               for name in df.index.names]
    return df.loc[tuple(indexer)] if len(df.shape) == 1 else df.loc[tuple(indexer),]


# Choose initial means with K-means method
def ChooseInitialMeans(X, k):
    #kmeans_missing(X, n_clusters, max_iter=500):
    distance, labels, centroids, X_hat = kmeans.kmeans_missing(X, k, max_iter=500)
    return centroids

# Returns the distance between an object x and a cluster mean c
def GetDist(x, c):
    dist = np.linalg.norm(x-c-boxl*np.around((x-c)/boxl))
    return dist

# Returns the distance matrix of each object to each cluster mean,
# and each row (i'th obj) of the matrix is ordered so that the dist are in ascending order
def SetDistMat(X, n, k, means):
    distmat_dtype = [('key',int), ('dist',float)]
    distmat = np.empty((n,k),dtype=distmat_dtype)
    for i in range(n):
        distmat[i,:] = [(c[0], GetDist(X[i], c[1])) for c in enumerate(means)]
        distmat[i,:] = np.sort(distmat[i,:], order='dist')
    return distmat

# Returns the priority list in which objects are ordered by ascending benefit 
# of best over worst assignment
def Get_plst(assigned, distmat, full, n, k):
    plst = []
    for i in range(n):
        if (i not in assigned):
            j = 0
            while j<k:
                if (not full[distmat[i,j][0]]):
                    bestkey = distmat[i,j][0]
                    mindist = distmat[i,j][1]
                    break
                else:
                    j += 1
            for j in range(k-1,-1,-1):
                if (not full[distmat[i,j][0]]):
                    maxdist = distmat[i,j][1]
                    break
            plst.append((i, bestkey, maxdist-mindist))
    plst.sort(key=lambda t:t[2])
    return plst

def InitialAssignment(distmat, cluster_size, n, k):
    clusters = {}
    full = np.zeros(k,dtype=bool) # a boolean array that records which clusters are full
    assigned = [] # a list of objects who has been assigned to a cluster
    plst = Get_plst(assigned, distmat, full, n, k)
    while (len(plst)):
        temp = plst.pop()
        try:
            if (len(clusters[temp[1]])<cluster_size):
                clusters[temp[1]].append(temp[0])
                assigned.append(temp[0])
            else:
                full[temp[1]] = True
                plst = Get_plst(assigned, distmat, full, n, k)
        except KeyError:
            clusters[temp[1]] = [temp[0]]
            assigned.append(temp[0])
    return clusters

def CalcMeans(X, oldmeans, clusters, k):
    means = np.zeros((k,3)) # should be three originally
    keys = sorted(clusters.keys())
    for key in keys:
        for i in clusters[key]:
            means[key] += X[i]-boxl*np.around((X[i]-oldmeans[key])/boxl)
        means[key] /= len(clusters[key])
        means[key] -= boxl*np.around(means[key]/boxl)
    return means

def SortObj(X, clusters, means, distmat):
    objlst = [] # list of objects ordered in asceding delta of the current 
                # assignment and the best possible alternate assignment
    keys = sorted(clusters.keys())
    for key in keys:
        for i in clusters[key]:
            currdist = GetDist(X[i],means[key])
            mindist = distmat[i,0][1]
            objlst.append((i, key, currdist-mindist))
    objlst.sort(key=lambda t:t[2], reverse=True)
    return objlst

"""Mostly useless in comparison to the rest of the code, but important"""
def Transfer(obj, clufrom, cluto, clusters):
    clusters[clufrom].remove(obj)
    clusters[cluto].append(obj)
    return clusters

model_matrix = np.matrix(modeling)
#def run_equal_size_kmeans(X, n, k, cluster_size):
# Set up the database of objects
#X = readdata.read_data(infl)
# Choose initial means with K-means

min_obs = 5
num_obs = len(model_matrix)
cluster_size = math.ceil(float(num_obs)/float(number_of_clusters))

Y = ChooseInitialMeans(model_matrix, k)
#means = ChooseInitialMeans(X, k)
# Set up initial clusters
#distmat = SetDistMat(X, n, k, means) 
#distmat_g = gower_distances(X, means)
#print distmat_g
#clusters = InitialAssignment(distmat, n, k, cluster_size)
## debug code
#keys = sorted(clusters.keys())
#for key in keys:
#    print("cluster %i:"%key)
#    print(clusters[key])
## end of debug
# Iteration step


#def gower_distances(X, Y=None, w=None):
w=None
"""
Computes the gower distances between X and Y

Read more in the :ref:`User Guide <metrics>`.

Parameters
----------
X : array-like, shape (n_samples, n_features)

Y : array-like, shape (n_samples, n_features)

Returns
-------
distances : ndarray, shape (n_samples, )

Notes
------
Gower is a similarity for categorical, boolean and numerical mixed data.
"""
X = model_matrix
X = pairwise.check_array(X, Y, dtype=np.object)
rows, cols = X.shape
dtypes = []
for col in range(cols):
    print col
    dtypes.append(type(X[0, col]))

# calculate the range and max values of numeric values for mixed data
ranges_of_numeric = [0.0] * cols
max_of_numeric = [0.0] * cols
for col in range(cols):
    print "looping columns for max and mixed: %s " % col
    if np.issubdtype(dtypes[col], np.number):
        max = np.nanmax(X[:, col].astype(dtypes[col])) + 0.0
        if np.isnan(max):
            max = 0.0

        max_of_numeric[col] = max

        min = np.nanmin(X[:, col].astype(dtypes[col])) + 0.0
        if np.isnan(min):
            min = 0.0

        ranges_of_numeric[col] = (1 - min / max,0)[max == 0]


# According the Gower formula, w is an attribute weight
if w is None:
    w = [1] * cols

yrows, ycols = Y.shape

dm = np.zeros((rows, yrows), dtype=object)
dmsout = []#np.zeros((rows, yrows), dtype=object)

for i in range(0, rows):
    print "starting loop: %s " % i
    j_start = i

    #for non square results
    if rows != yrows:
        j_start = 0

    for j in range(j_start, yrows):
        xi = X[i]
        xj = Y[j]
        sum_sij = 0.0
        sum_wij = 0.0
        for col in range(cols):
            value_xi = xi[col]
            value_xj = xj[col]
            if np.issubdtype(dtypes[col], np.number):
                if (max_of_numeric[col] != 0):
                    value_xi = value_xi / max_of_numeric[col]
                    value_xj = value_xj / max_of_numeric[col]
                else:
                    value_xi = 0
                    value_xj = 0

                if ranges_of_numeric[col] != 0 :
                    sij = abs(value_xi - value_xj) / ranges_of_numeric[col]
                else:
                    sij=0
                wij = (w[col], 0)[np.isnan(value_xi) or np.isnan(value_xj)]
            else:
                sij = (1.0, 0.0)[value_xi == value_xj]
                wij = (w[col], 0)[value_xi is None and value_xj is None]
            sum_sij += (wij * sij)
            sum_wij += wij

        if sum_wij!=0:
            dm[i,j] = (j,(sum_sij / sum_wij))
            #dmsout[i][j] = (col, (sum_sij / sum_wij))
            
            if j < rows and i<yrows :
                dm[j,i] = dm[i,j]
                #dmsout[i][j] = dmsout[i][j]

#    return dm


In [ ]:
dm

In [ ]:
distmat = SetDistMat(model_matrix, n, k, Y)
distmat

In [ ]:


#     for iter in range(max_iter):
#         active = 0 # indicate the number of transfers in the current iteration
#         tranlst = (-1)*np.ones(k, dtype='int') # set up transfer list for each cluster
#         # Compute the cluster means
#         oldmeans = means.copy()
#         means = CalcMeans(X, oldmeans, clusters)
#         # Get statistics about the clustering
#         #ClusterStat(X, means, clusters)
#         ## debug code
#         #print("old means:")
#         #print(oldmeans)
#         #print("new means:")
#         #print(means)
#         ## end of debug
#         # For each object, compute the distances to the cluster means
#         distmat = SetDistMat(X, n, k, means)
#         # Sort objects based on the delta of the current assignment and the best 
#         # possible alternate assignment
#         objlst = SortObj(X, clusters, means, distmat)
#         ##debug code
#         #print(objlst)
#         ##return
#         #end of debug
#         # For each element by prioty:
#         while (len(objlst)):
#             (i, key, temp) = objlst.pop()
#             obj2key = GetDist(X[i], means[key])
#             transferred = False #record if any transfering has occured to i 
#             if (key == distmat[i,0][0]):
#                 ##debug
#                 #print("%i is already the opt cluster for obj %i. no transfer"%(clu, i))
#                 ##end of debug
#                 continue
#             # For each other clusters by element gain:
#             else:
#                 for j in range(k):
#                     clu = distmat[i,j][0] # the key of another cluster
#                     objgain = obj2key - distmat[i,j][1] # gain by transfering i from cluster key to clu
#                     if (clu==key): # already in the cluster
#                         continue
#                     if (len(clusters[clu]) < cluster_size):
#                         active += 1
#                         transferred = True
#                         clusters = Transfer(i, key, clu, clusters)
#                         ##debug
#                         #print("cluster %i not full. transfer obj %i from cluster %i to it."%(clu, i, key))
#                         ##end of debug
#                         break
#                     elif (tranlst[clu] != -1): # if the tranlst of another cluster is not empty
#                         # distance between the obj in the tranlst and the current cluster
#                         tran2key = GetDist(X[tranlst[clu]], means[key])
#                         tran2clu = GetDist(X[tranlst[clu]], means[clu])
#                         # gain by transfering the obj in tranlst from cluster clu to key
#                         trangain = tran2clu - tran2key
#                         if (objgain + trangain > 0): # transfer if the sum of gains are positive, ie net gain
#                             active += 2
#                             transferred = True
#                             clusters = Transfer(i, key, clu, clusters)
#                             clusters = Transfer(tranlst[clu], clu, key, clusters)
#                             ##debug
#                             #print("obj %i is transfered from cluster %i to %i"%(i, key, clu))
#                             #print("obj %i is transfered from cluster %i to %i"%(tranlst[clu], clu, key))
#                             #print("objgain: %f, trangain: %f"%(objgain, trangain))
#                             ##end of debug
#                             tranlst[clu] = -1 # reset the tranlst to empty
#                             break
#                 if (not transferred):
#                     tranlst[key] = i
#                     ##debug
#                     #print("add obj %i in cluster %i to the transfer list"%(i, key))
#                     ##end of debug
#         # nothing is transferred during this iteration, return the clustering result
#         if (not active):
#                 break
#         #debug code
#         print("number of transfers in iter %i: %i\n"%(iter+1, active))
#         #end of debug
#     print("K-means clustering converged in %d iterations!\n"%(iter+1))
#     # Output the clustering results
#     WriteResult(outfl, X, means, clusters, k)
#     ClusterStat(X, means, clusters, k)


In [ ]:
modeling['CITYMARKET']

In [ ]:
# This section calculates the clusters for the hotels

total_frame = pd.DataFrame()
distance_frame = pd.DataFrame()

for market in markets:
    for job in positions:
        temp_frame = d_ci_v2[d_ci_v2['CITYMARKET'] == market]
        temp_frame = temp_frame[temp_frame['POSITION'] == job]
        if temp_frame.shape[0] > 1:
            temp_frame = temp_frame.fillna(np.nan)
            temp_frame['clusters'] = ''
            print market + " " + job + " "# + str(temp_frame['CREATED_cur_base_pay_hrs'])

            #this needs to be in a function like so:
            #cluster_positions(temp_frame)
            pcode = temp_frame['PROPERTY_CODE'].values
#             f1 = np.linalg.norm(temp_frame['ANSWER_num_rooms'].astype(float).values)
#             f2 = np.linalg.norm(temp_frame['CREATED_revpar'].astype(float).values)
#             f3 = np.linalg.norm(temp_frame['ANSWER_num_employees_pos'].astype(int).values)

            temp_frame['rank_sum'] = .5*f1 + .5*f3
            temp_frame = temp_frame.sort_values(by=['rank_sum'])
            
            temp_matrix = np.matrix((f1,f2,f3))
            

            #print temp_frame['ANSWER_num_rooms'].astype(float).values
            #test = kmeans.run_equal_size_kmeans(temp_matrix, num_obs, number_of_clusters, num_obs_per_cluster)
#             temp_frame_km = kmeans.kmeans_missing(temp_matrix, number_of_clusters, max_iter=500)
#             means = kmeans.ChooseInitialMeans(temp_matrix)
#             distmat = kmeans.SetDistMat(temp_matrix, means)
#             clusters = kmeans.InitialAssignment(distmat, num_obs_per_cluster, num_of_clusters)

            min_obs = 5
            num_obs = temp_frame.shape[0]
            num_obs_per_cluster = math.ceil(float(num_obs)/float(number_of_clusters))
            num_obs_mod = num_obs_per_cluster % min_obs
            
            print "number of observations: %s" % num_obs
            print "number of obs per cluster: %s" % num_obs_per_cluster

            if num_obs < 5:
                temp_frame['clusters'] = 'No'
            elif num_obs >= 5 and num_obs < 10:
                temp_frame['clusters'] = '0'
            elif num_obs > 10:
                for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // num_obs_per_cluster):
                    temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 10:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 11:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs+1)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs >= 12 and num_obs <= 14:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs+2)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 15:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 16:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (num_obs_per_cluster+2)):
#                     temp_frame.ix[df.index,'clusters'] = g                       
#             elif num_obs > 17:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (num_obs_per_cluster)):
#                     temp_frame.ix[df.index,'clusters'] = g
                        
            total_frame = total_frame.append(temp_frame)
            
            if num_obs > 11:
                pass
                #print temp_frame[['PROPERTY_NAME','clusters']]
    else:
        total_frame = total_frame


In [ ]:
f2

# This section will be used to calculate summary statistics, diagnostics, and model outputs

In [ ]:
jobs_frame = total_frame
jobs_frame['CREATED_cur_base_pay_hrs'] = jobs_frame['CREATED_cur_base_pay_hrs'].astype(float)
j_frame = jobs_frame.groupby(['CITYMARKET','POSITION','clusters'])['CREATED_cur_base_pay_hrs'].describe(include='all').reset_index()
j_frame = pd.pivot_table(j_frame, index = ['CITYMARKET','POSITION','clusters'], columns = ['level_3'], aggfunc=np.sum).reset_index()
j_frame.columns = ['CITYMARKET','POSITION','clusters','25','50','75','count','max','mean','min','std']
j_frame = j_frame.sort_values(by=['CITYMARKET','POSITION', 'clusters','count'])

hotels_frame = total_frame[['PROPERTY_CODE','PROPERTY_NAME','CITYMARKET','STATE','ZIP','POSITION','clusters']]

In [ ]:
############################
# This is to be turned into a function which creates the output for reporting. 
# an xlsx which will contain summary and detailed info for a 'top market' run. 
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('ww_top_' + str(number_of_markets) + '_markets_output.xlsx', engine='xlsxwriter')

for market in markets:
    
    jobs_sheet = j_frame[j_frame['CITYMARKET'] == market]
    jobs_sheet_name = market + "_" + "wages"
    jobs_sheet.to_excel(writer, sheet_name = jobs_sheet_name, encoding='utf8')

    hotels_sheet = hotels_frame[hotels_frame['CITYMARKET'] == market]
    hotels_sheet_name = market + "_" + "hotels"
    hotels_sheet.to_excel(writer, sheet_name = hotels_sheet_name, encoding='utf8')
    

# Close the Pandas Excel writer and output the Excel file.
writer.save()